# HW2 Solutions


## Import libraries


##### `nltk`

- **Description**: `Natural Language Toolkit`, I use it for tokenization, stemming , remove stop words and lemmatizer.

##### `glob`

- **Description**: Just for reading documents name in ./docs directory

##### `string`

- **Description**: I used from punctuations attribute on this package


In [1]:
import nltk
import glob
import string
# nltk.download('stopwords')
# nltk.download('wordnet')

## Load Documents


The `LoadDocuments` class is responsible for loading and managing a collection of documents. If you want to change the document path please cgange path variable.

##### Properties

- **_`DOC_ID_MAPPER`_** _(class variable)_: A dictionary mapping document names to their respective IDs, Be care full, becuse it depends on your documents order and docIds started from zero.

##### Methods

##### `__init__(self)`

- **Description**: Initializes a `LoadDocuments` object with a default path to load text that you can change it base on your docs directory path and an empty `documentCollection`. that is a dictionary.

##### `Load(self)`

- **Description**: Loads text documents from the specified path and populates the `documentCollection` property.

##### `buildDocumentIDMapper(self)`

- **Description**: Builds a document ID mapper by associating each document's name with a unique ID and populates the `DOC_ID_MAPPER` dictionary, I will print this mapper data for checking my results base on this mapper.


In [2]:
class LoadDocuments:

    DOC_ID_MAPPER = {}

    def __init__(self) -> None:
        self.path = "./docs/*.txt"
        self.documentCollection = {}

    def Load(self)->None:
        documents = glob.glob(self.path)
        for documentName in documents :
            document = open(documentName, "r", encoding = 'cp1252')
            self.documentCollection[documentName] = document.read()

    def buildDocumentIDMapper(self)->None:
        files = glob.glob(self.path)
        for docId, documentName in enumerate(files):
            self.DOC_ID_MAPPER[documentName] = docId
            print(docId , ' -> ', documentName[7:])  

## Document Preprocessing


The `DocumentPreProcessor` class extends the functionality of the `LoadDocuments` class by providing methods to preprocess text documents. These methods perform various text processing tasks, such as converting text to lowercase, tokenization, removing punctuations, removing stop words, stemming, and lemmatization.

##### Properties

- **_`correctWords`_** _(class variable)_: As question said we assume that terms in our documents are correst, so I will add all terms in my documents ro this array because I need it for miss spelling

#### Methods

##### `__init__(self)`

- **Description**: Initializes a `DocumentPreProcessor` object by calling the constructor of the base class `LoadDocuments`.

##### `convertToLower(self)`

- **Description**: Converts all text in the loaded documents to lowercase.

##### `tokenizer(self)`

- **Description**: Tokenizes the text in the loaded documents using a regular expression pattern.

##### `removePunctuations(self)`

- **Description**: Removes punctuations from the tokenized text in the loaded documents.

##### `removeStopWords(self)`

- **Description**: Removes stop words from the tokenized text in the loaded documents using NLTK's English stop words list.

##### `buildCorrectWordsList(self)`

- **Description**: Append all terms in all documents to this list a correct word hub

##### `stemming(self)`

- **Description**: Applies stemming to the tokenized text in the loaded documents using the Porter Stemmer algorithm from NLTK.

##### `lemmatizer(self)`

- **Description**: Applies lemmatization to the tokenized text in the loaded documents using NLTK's WordNet Lemmatizer.


In [3]:
class DocumentPreProcessor(LoadDocuments):

    correctWords = []

    def __init__(self)->None:
        super().__init__()

    def convertToLower(self)->None:
        for documentName, document in self.documentCollection.items():
            self.documentCollection[documentName] = document.lower()

    def tokenizer(self)->None:
        pattern = r'\d{1,3}(?:,\d{3})*(?:\.\d+)?|\w+'
        for documentName, document in self.documentCollection.items():
            self.documentCollection[documentName] = nltk.tokenize.regexp_tokenize(document, pattern)

    def removePunctuations(self)->None:
        for documentName, document in self.documentCollection.items():
            for ind, term in enumerate(document):
                self.documentCollection[documentName][ind] = "".join([i for i in term if i not in string.punctuation])

    def removeStopWords(self)->None:
        stopwords = nltk.corpus.stopwords.words('english')
        for documentName, document in self.documentCollection.items():
            self.documentCollection[documentName] = [i for i in document if i not in stopwords]

    def buildCorrectWordsList(self)->list:
        self.correctWords = [term for _,document in self.documentCollection.items() for term in document]
        
    def stemming(self)->None:
        porter_stemmer = nltk.stem.porter.PorterStemmer()
        for documentName, document in self.documentCollection.items():
            self.documentCollection[documentName] = [porter_stemmer.stem(term) for term in document]

    def lemmatizer(self)->None:
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
        for documentName, document in self.documentCollection.items():
            self.documentCollection[documentName] = [wordnet_lemmatizer.lemmatize(term) for term in document]

## Inverted Index


The `InvertedIndex` class will generate a invertedIndexex that maps terms to the documents that contain them, I will save all positions for proximate search .

#### Properties

- **`invertedIndex`**: A dictionary representing the inverted index where terms are mapped to document IDs and their positions.

#### Methods

##### `__init__(self)`

- **Description**: Initializes an `InvertedIndex` object and call base class constructor.

##### `buildInvertedIndex(self)`

- **Description**: Builds the inverted index by iterating through preprocessed documents and mapping terms to document IDs and their positions.


In [4]:
class InvertedIndex(DocumentPreProcessor):

    def __init__(self) -> None:
        super().__init__()
        self.invertedIndex = {}

    def buildInvertedIndex(self)->None:
        for documentName, document in self.documentCollection.items():
            for ind, term in enumerate(document):
                if term not in self.invertedIndex:
                    self.invertedIndex[term] = {}

                if(self.DOC_ID_MAPPER[documentName] not in self.invertedIndex[term]):
                    self.invertedIndex[term][self.DOC_ID_MAPPER[documentName]] = []

                self.invertedIndex[term][self.DOC_ID_MAPPER[documentName]].append(ind)
                  

## Trie Tree


Represents a Trie data structure for storing and querying words.

#### TrieNode Properties

- `char`: Character stored in the node.
- `isEnd`: Boolean value indicating if the node represents the end of a word.
- `counter`: Integer count representing the number of words with the same prefix.
- `child`: Dictionary containing child nodes.

#### Properties

- `root`: The root node of the Trie.

#### Methods

##### `__init__(self)`

Constructor method. Initializes a Trie object with an empty root node.

##### `insert(self, string: str)`

Inserts a word into the Trie.

##### `query(self, x: str)`

Queries the Trie for words matching the given prefix `x`. Returns a list of matching words.


In [5]:
class TrieNode:
    def __init__(self, char) -> None:
        self.char = char
        self.isEnd = False
        self.counter = 0
        self.child = {}

class Trie():
    def __init__(self) -> None:
        self.root = TrieNode("")

    def insert(self, string :str)->None:
        node = self.root

        for char in string:
            if char in node.child:
                node = node.child[char]
            else:
                new_node = TrieNode(char)
                node.child[char] = new_node
                node = new_node
        node.isEnd = True
        node.counter +=1

    def dfs(self, node: TrieNode, prefix:str)->None:
        if node.isEnd:
            self.output.append(prefix + node.char)
        
        for child in node.child.values():
            self.dfs(child, prefix + node.char)

    def query(self, x):
        self.output = []
        node = self.root
        
        for char in x:
            if char in node.child:
                node = node.child[char]
            else:
                return []

        self.dfs(node, x[:-1])
        
        return self.output

## Permuterm Index


The `PermutermIndex` class extends `DocumentPreProcessor` and provides functionality for building a Permuterm index and searching for wildcard queries in a collection of documents.

#### Properties

- `permutermIndex`: Dictionary containing permuterm rotations of words.
- `trie`: Trie data structure for efficient wildcard query handling.

#### Methods

##### `__init__(self)`

Constructor method. Initializes a `PermutermIndex` object with an empty `permutermIndex` dictionary and a Trie.

##### `buildPermuteIndex(self)`

Builds the permuterm index using the corrected words obtained from `DocumentPreProcessor`. Populates the `permutermIndex` dictionary and inserts permuterm rotations into the Trie.

##### `searchWildcard(self, query: str)`

Searches for wildcard queries in the permuterm index. Handles queries with one or two '\*' symbols. Returns a list of matching words.


In [6]:
class PermutermIndex(DocumentPreProcessor):

    def __init__(self):
        super().__init__()
        self.permutermIndex = {}
        self.trie = Trie()

    def buildPermuteIndex(self)->None:
        for term in self.correctWords:
            term = term + "$"
            rotations = [term[i:] + term[:i] for i in range(len(term))]
            for rotation in rotations:
                if rotation not in self.permutermIndex:
                    self.permutermIndex[rotation] = []
                self.permutermIndex[rotation].append(term[:-1])

                self.trie.insert(rotation)            

    def searchWildcard(self, query:string):
        if(query.count("*") == 1):
            query = query + "$"
            prefix, suffix = query.split("*")
            query = suffix + prefix
            res = self.trie.query(query)
            result = []
            for term in res:
                if term in self.permutermIndex:
                    result.extend(self.permutermIndex[term])
            return result

        elif(query.count("*") == 2):
            query = query + "$"
            prefix, mid , suffix = query.split("*")
            query = suffix + prefix
            res = self.trie.query(query)
            result = []
            for term in res:
                if term in self.permutermIndex :
                    for word in self.permutermIndex[term]:
                        if mid in word[len(prefix)-1:len(term)-len(suffix)]:
                            result.append(word)
            return list(set(result))

## Indexes:


The `Indexes` class will generate all indexes, it is a bridge between all types of indexes and will generate all of them.

#### Methods

##### `__init__(self)`

- **Description**: Initializes base class constructor.

##### `build(self) -> None`

- **Description**: Initiates the preprocessing steps by loading documents, building document ID mapper, converting to lowercase, tokenizing, removing punctuations, removing stop words, and finally, building inverted index and wildCard index


In [7]:
class Indexes(PermutermIndex, InvertedIndex):
    def __init__(self):
        super().__init__()
    
    def build(self)->None:
        self.Load()
        self.buildDocumentIDMapper()
        self.convertToLower()
        self.tokenizer()
        self.removePunctuations()
        self.removeStopWords()
        self.buildCorrectWordsList()
        # self.stemming()
        # self.lemmatizer()
        self.buildInvertedIndex()  
        self.buildPermuteIndex()

## Spelling Correction


The `SpellCorrection` class provides methods for correcting misspelled words using the Levenshtein distance algorithm.

#### Methods

##### `__init__(self)`

Constructor method. Initializes a `SpellCorrection` object.

##### `levenshteinDistanceCal(self, s1: str, s2: str) -> int`

Calculates the Levenshtein distance between two strings `s1` and `s2`.

##### `getCorrectWord(self, input: str, correctWords: list) -> str`

Finds the correct word from a list of `correctWords` that is closest to the `input` string in terms of Levenshtein distance. for this I return just the first one but we can return a list of nearst words ...


In [8]:
class SpellCorrection:
    def __init__(self) -> None:
        pass

    def levenshteinDistanceCal(self, s1: str, s2: str)->int:
        m = [[0 for _ in range(len(s2)+1)] for _ in range(len(s1)+1)]

        for i in range(1 , len(s1)+1):
            m[i][0] = i
        for j in range (1, len(s2)+1):
            m[0][j] = j
        for i in range(1, len(s1)+1):
            for j in range(1, len(s2)+1):
                tmp = 1
                if s1[i-1] == s2[j-1] :
                    tmp = 0
                m[i][j] = min(m[i-1][j-1]+ tmp, m[i-1][j]+1, m[i][j-1] + 1)
       
        return m[len(s1)][len(s2)]

    def getCorrectWord(self, input :str, correctWords: list)->str:
        levenshteinDistances = [self.levenshteinDistanceCal(s1, input) for s1 in correctWords]   
        minDistance = min(levenshteinDistances)
        return [correctWords[ind] for ind, dis in enumerate(levenshteinDistances) if dis == minDistance][0]

## Query Processing


The `QueryPreProcessing` class provides methods to preprocess search queries. It is like document preprocessing because I want to make them similar but I don't remove punctuations.

#### Properties

- **`query`**: the Query string that I gave from user
- **`queryType`**: A string indicating the type of query (`AND`, `OR`, `NOT`, or `NEAR`), I keep my query type and then I will remove it from my query.
- **`maxDistance`**: An integer representing the maximum distance for `NEAR` queries.

#### Methods

##### `__init__(self, query:str)`

- **Description**: Initializes a `QueryPreProcessing` object with the provided query string.

##### `determineQueryType(self)`

- **Description**: Determines the query type based on the query string and sets the `queryType` and `maxDistance` properties.

##### `convertToLower(self)`

- **Description**: Converts all query terms to lowercase.

##### `removePunctuations(self)`

- **Description**: Removes punctuations from query terms.

##### `removeQueryEXP(self)`

- **Description**: Removes specific query expressions (`AND`, `OR`, `NOT`, and `NEAR`) from the query.

##### `spellCorrection(self)`

- **Description**: Check spell correction and I will use my SpellCorrection to remove miss spelling

##### `stemming(self)`

- **Description**: Stems query terms using the Porter Stemmer algorithm.

##### `lemmatizer(self)`

- **Description**: Lemmatizes query terms using the WordNet Lemmatizer.

##### `execute(self)`

- **Description**: Executes the entire preprocessing for generating a clean query.


In [9]:
class QueryPreProcessing:

    ALL_QUERY_TYPES = [
        "and",
        "or",
        "not",
        "near",
        "spellCorrection",
        "wildCard"
    ]

    def __init__(self, query:str, correctWords : list) -> None:
        self.correctWords = correctWords
        self.query = query.split(" ")
        self.queryType = ""
        self.maxDistance = 0

    def determineQueryType(self)->None:
        if self.query[0] == "NOT":
            self.queryType = "NOT"
        elif len(self.query) == 1 and "*" in self.query[0]:
            self.queryType = "wildCard"    
        elif len(self.query) == 1 :
            self.queryType = "spellCorrection" 
        elif self.query[1] == "AND":
            self.queryType = "AND"
        elif self.query[1] == "OR":
            self.queryType = "OR"
        elif self.query[1].startswith("NEAR"):
            self.queryType = "NEAR"
            self.maxDistance = int(self.query[1].split("/")[1])   
        else:    
            self.queryType = "spellCorrection"
        

    def convertToLower(self)->None:
            for ind, term in enumerate(self.query):
                self.query[ind] = term.lower()

    def removePunctuations(self) ->None:
        for ind, term in enumerate(self.query):
            punctuations = r"""!"#$%&'()+,-./:;<=>?@[\]^_`{|}~""" # I removed * for wildcard
            self.query[ind] ="".join([i for i in term if i not in punctuations])           

    def removeQueryEXP(self)->None:
        self.query = [term for term in self.query if term not in self.ALL_QUERY_TYPES and not term.startswith("NEAR")]    

    def spellCorrection(self)->None:
        spellCorrection = SpellCorrection()
        for ind, term in enumerate(self.query):
            if "*" not in term:
                self.query[ind] = spellCorrection.getCorrectWord(term, self.correctWords)  

    def stemming(self)->None:
        porter_stemmer = nltk.stem.porter.PorterStemmer()
        self.query = [porter_stemmer.stem(term) for term in self.query]

    def lemmatizer(self)->None:
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
        self.query = [wordnet_lemmatizer.lemmatize(term) for term in self.query]  

    def execute(self)->None:
        self.determineQueryType()
        self.convertToLower()
        self.removePunctuations()  
        self.removeQueryEXP()
        self.spellCorrection()
        # self.stemming()
        # self.lemmatizer()

## Utils


The `Utils` class contains utility methods for set operations like intersection and union and proximity distance calculations that is a little custom function(maybe I should change it's name).

#### Methods

##### `interSect(ls: list)`

- **Description**: Calculates the intersection of two lists.

##### `union(ls: list)`

- **Description**: Calculates the union of two lists.

##### `complement(ls: list)`

- **Description**: Calculates the complement of two sets.

##### `proximateDistanceCal(ls: list, maxDistance: int)`

- **Description**: Calculates proximate distances within the given maximum distance between indexes.


In [10]:
class Utils:
    def __init__(self) -> None:
        pass

    def interSect(ls: list)->list:
        return list(set([i for i in ls[0] if i in ls[1]]))
    
    def union(ls :list)->list:
        return list(set(ls[0] + ls[1]))
    
    def complement(ls: list)->list:
        return list(set([i for i in ls[1].values() if i not in ls[0]]))
    
    def proximateDistanceCal(ls: list, maxDistance: int)->list:
        result = []
        for docId, indexes in ls[0].items():
            if(docId in ls[1]):
                for i in range (len(indexes)-1):
                    if(abs(indexes[i] - indexes[i+1]) <= maxDistance + 1):
                        result.append(docId)
                for i in range(len(ls[1][docId])-1):
                    if(abs(ls[1][docId][i] - ls[1][docId][i+1]) <= maxDistance + 1):
                        result.append(docId)     

        return list(set(result))                        


## IR System


The `IR` class is my information retrieval system that will generate InvertedInxe and will get your query and generate output, If you want a IR system that get many queries and be up for a while you can put start function in while.

#### Methods

##### `booleanSearch(query: QueryPreProcessing)`

- **Description**: Performs boolean search operations (AND, OR, NOT) based on the provided query.

##### `proximateSearch(query: QueryPreProcessing)`

- **Description**: Performs proximate search operations based on the provided query and maximum distance.

##### `wildcardSearch(self, queryTerm: str)`

- **Description**: Performs serch on a term that is a wildcard query like $A^*B^*C$

##### `search(query: QueryPreProcessing)`

- **Description**: Determines the type of search operation (boolean or proximate) based on the query type and performs the search.

##### `start()`

- **Description**: Initiates the information retrieval process by taking user input for the query, processing it, and displaying the resulting document IDs.


In [11]:
class IR:
    def __init__(self) -> None:
        self.indexes = Indexes()
        self.indexes.build()
        self.utils = Utils()

    def booleanSearch(self, query: QueryPreProcessing)->list:
        result_docs = []
        for term in query.query:
            if term.count("*") == 0:
                if term in self.indexes.invertedIndex:
                    result_docs.append(list(self.indexes.invertedIndex[term].keys()))
                else: result_docs.append([])
            else:
                result_docs.append(self.wildcardSearch(term))  
         
        if(query.queryType == "AND"): return Utils.interSect(result_docs)
        elif (query.queryType == "OR") : return Utils.union(result_docs)
        elif(query.queryType == "NOT") : 
            result_docs.append(InvertedIndex.DOC_ID_MAPPER)
            return Utils.complement(result_docs)

    def proximateSearch(self, query: QueryPreProcessing)->list:
        result_docs = []
        for term in query.query:
            if term in self.indexes.invertedIndex:
                result_docs.append(self.indexes.invertedIndex[term])

        return Utils.proximateDistanceCal(result_docs, query.maxDistance)
    
    def wildcardSearch(self, queryTerm: str)->list:  
        resultTerms = self.indexes.searchWildcard(queryTerm)
        result_docs = []
        for term in resultTerms:
            if term in self.indexes.invertedIndex:
               result_docs.extend(self.indexes.invertedIndex[term])
        return list(set(result_docs))
            
    def search(self, query: QueryPreProcessing)-> list:
        if query.queryType == "spellCorrection":
            return query.query
        elif query.queryType == "wildCard":
            return self.wildcardSearch(query.query[0])
        elif(query.queryType != "NEAR"):
            return self.booleanSearch(query)
        else:
            return self.proximateSearch(query)   
        
    def start(self)->None:
        while True:
            query = input("Enter Your Query: '\n'") 
            if(query == "end"):
                break  
            print('\n') 
            print("Input query : " , query)
            query = QueryPreProcessing(query, self.indexes.correctWords)
            query.execute()
            print("query Type: ", query.queryType)
            print("query terms after pre process: ", query.query)
            print("Result : " , self.search(query))

In [12]:
IR = IR()
IR.start()

0  ->  Jerry Decided To Buy a Gun.txt
1  ->  Rentals at the Oceanside Community.txt
2  ->  Gasoline Prices Hit Record High.txt
3  ->  Cloning Pets.txt
4  ->  Crazy Housing Prices.txt
5  ->  Man Injured at Fast Food Place.txt
6  ->  A Festival of Books.txt
7  ->  Food Fight Erupted in Prison.txt
8  ->  Better To Be Unlucky.txt
9  ->  Sara Went Shopping.txt
10  ->  Freeway Chase Ends at Newsstand.txt
11  ->  Trees Are a Threat.txt
12  ->  A Murder-Suicide.txt
13  ->  Happy and Unhappy Renters.txt
14  ->  Pulling Out Nine Tons of Trash.txt




Input query :  jer*y
query Type:  wildCard
query terms after pre process:  ['jer*y']
Result :  [0]


Input query :  n*b*y
query Type:  wildCard
query terms after pre process:  ['n*b*y']
Result :  [11, 6]


Input query :  jer*y
query Type:  wildCard
query terms after pre process:  ['jer*y']
Result :  [0]


## Report:


### Steps:

##### 1. Initially, I retrieved my previous exercise codes, forming the foundation of the project.

##### 2. I utilized the document preprocessor to generate a comprehensive list of correct words from the document corpus.

##### 3. A robust spell correction mechanism was implemented using the Levenshtein distance algorithm. This ensured precise word correction based on the list of correct words.

##### 4. Added new query types that we want.

##### 5. A trie tree data structure was developed to facilitate the Permuterm Index, enabling seamless handling of wildcard queries.

##### 6. The Permuterm Index was successfully implemented, addressing challenges posed by wildcard queries with multiple '\*' symbols. Innovative solutions were applied in algorithm design.

##### 7. All components were integrated, necessitating adjustments in the Information Retrieval (IR) system to handle wildcard and spell correction queries seamlessly.


## References:


1. Our course slides
2. https://www.nltk.org/
3. https://chat.openai.com/
4. https://nlp.stanford.edu/IR-book/html/htmledition/permuterm-indexes-1.html#:~:text=We%20refer%20to%20the%20set,query%20becomes%20n%24m*
